<a href="https://colab.research.google.com/github/Rellendula26/PersonalPortfolio-RitvikEllendula-/blob/Highlights/Count_Coach_V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
!pip install librosa soundfile spotipy python-dotenv


In [11]:
import getpass

SPOTIPY_CLIENT_ID = getpass.getpass("Enter your Spotify Client ID: ")
#these two lines get your spotify api keys without returning the input outwards like the normal input function would
SPOTIPY_CLIENT_SECRET = getpass.getpass("Enter your Spotify Client Secret: ")
#so user types in spotify api credentials

Enter your Spotify Client ID: ··········
Enter your Spotify Client Secret: ··········


In [12]:
import spotipy
#python's wrapper for spotify api --> helps search for songs, track features, audio analysis
from spotipy.oauth2 import SpotifyClientCredentials
#used for backend scripts

#takes api keys and creates an authentication manager
def create_spotify_client(client_id, client_secret):
    client_id = client_id.strip()
    client_secret = client_secret.strip()
    auth_manager = SpotifyClientCredentials(
        client_id=client_id,
        client_secret=client_secret
    )
    return spotipy.Spotify(auth_manager=auth_manager)
    #authentication step for fetching data

In [14]:
sp = create_spotify_client(SPOTIPY_CLIENT_ID, SPOTIPY_CLIENT_SECRET)
#spotify client that is used in order to fetch any tracks from spotify

result = sp.search(q="blinding lights", type="track", limit=1)
#args are search query, type is like song, album, etc, limit says first one
result["tracks"]["items"][0]["name"], result["tracks"]["items"][0]["artists"][0]["name"]
#then this line returns two things the track's name and the artist



('Blinding Lights', 'The Weeknd')

In [19]:
import librosa
import numpy as np
import pandas as pd

def run_count_coach_from_file(path, sr_target=None):
    # 1. Load audio
    y, sr = librosa.load(path, sr=sr_target, mono=True)
    print(f"Loaded: {path}, duration = {len(y)/sr:.2f}s, sr = {sr}")

    # 2. Beat tracking (tempo + beat frames)
    tempo, beat_frames = librosa.beat.beat_track(y=y, sr=sr, units='frames')

    # 👉 tempo might be an ndarray, so force it to a float
    tempo_val = float(np.atleast_1d(tempo)[0])

    # convert frames → times
    beat_times = librosa.frames_to_time(beat_frames, sr=sr)
    print(f"Tempo: {tempo_val:.1f} BPM | Beats detected: {len(beat_times)}")

    # 3. Onset strength as intensity proxy
    onset_env = librosa.onset.onset_strength(y=y, sr=sr)

    # make sure frame indices are valid + int
    beat_frames = beat_frames.astype(int)
    valid_idx = beat_frames < len(onset_env)
    beat_frames = beat_frames[valid_idx]
    beat_times = beat_times[valid_idx]

    intensities = onset_env[beat_frames]

    # 4. Normalize intensity 0–1
    if intensities.max() != intensities.min():
        intensities_norm = (intensities - intensities.min()) / (intensities.max() - intensities.min())
    else:
        intensities_norm = np.zeros_like(intensities)

    # 5. Approximate beat durations from time diffs
    if len(beat_times) > 1:
        durations = np.diff(
            np.append(
                beat_times,
                beat_times[-1] + (beat_times[-1] - beat_times[-2])
            )
        )
    else:
        durations = np.array([0.0])

    # 6. Build DataFrame
    df = pd.DataFrame({
        "beat_index": np.arange(len(beat_times)),
        "start_time_sec": beat_times,
        "duration_sec": durations,
        "intensity": intensities_norm
    })

    # 7. Add 8-count structure
    df["count"] = (df["beat_index"] % 8) + 1
    df["eight_count_group"] = (df["beat_index"] // 8) + 1

    return df


In [20]:
from google.colab import files
uploaded = files.upload()

audio_path = list(uploaded.keys())[0]
audio_path

Saving Grover_Washington_-_Just_The_Two_Of_Us_Ft._Bill_Withers_(mp3.pm).mp3 to Grover_Washington_-_Just_The_Two_Of_Us_Ft._Bill_Withers_(mp3.pm) (1).mp3


'Grover_Washington_-_Just_The_Two_Of_Us_Ft._Bill_Withers_(mp3.pm) (1).mp3'

In [21]:
df = run_count_coach_from_file(audio_path)
df.head(20)

Loaded: Grover_Washington_-_Just_The_Two_Of_Us_Ft._Bill_Withers_(mp3.pm) (1).mp3, duration = 241.45s, sr = 44100
Tempo: 95.7 BPM | Beats detected: 361


,beat_index,start_time_sec,duration_sec,intensity,count,eight_count_group
0,0,0.417959,0.626939,0.882724,1,1
1,1,1.044898,0.650159,0.328667,2,1
2,2,1.695057,0.603719,0.578915,3,1
3,3,2.298776,0.615329,0.470251,4,1
4,4,2.914104,0.650159,1.000000,5,1
5,5,3.564263,0.626939,0.463001,6,1
6,6,4.191202,0.615329,0.533608,7,1
7,7,4.806531,0.626939,0.470464,8,1
8,8,5.433469,0.626939,0.675419,1,2
9,9,6.060408,0.615329,0.311926,2,2
